In [2]:
import pandas as pd
import numpy as np
import pvlib
from pvlib.location import Location
import sympy as sm
from scipy import integrate

def inner(x1: sm.Matrix,x2: sm.Matrix):
    '''
    Computes the inner product of two vectors of same length.
    '''

    return x1.dot(x2, conjugate_convention = 'right')

sm.MutableDenseMatrix.inner = inner
sm.ImmutableDenseMatrix.inner = inner
L = 1.755
B = 1.038
Area = L * B * 25 # 25 paneler
maxFlux = 1000
maxEffect = 365
A = 0.5
S0 = 1100
S0A = S0 * A
effectPerFlux = maxEffect/maxFlux

In [7]:
def solar_panel_projection(θ_s, ϕ_s, θ_p, ϕ_p):
    results = []
    for i in range(0,len(θ_s)):
        us = sm.Matrix([-np.sin(θ_s[i]) * np.cos(ϕ_s[i]),-np.sin(θ_s[i]) * np.sin(ϕ_s[i]), -np.cos(θ_s[i])])
        up = sm.Matrix([-1*np.sin(θ_p[i]) * np.cos(ϕ_p[i]), -np.sin(ϕ_p[i])*np.sin(θ_p[i]), -np.cos(θ_p[i])])
        #display(us, up)
        inprod = (inner(us,up))
        simpinprod = inprod.simplify()
        if(simpinprod < 0):
            results.append(0.0)
        else:
            results.append(simpinprod)
    return results

def fluxMinuteEntireDay(day, theta_p, phi_p):
    S0A=550
    L = 1.755
    B = 1.038

    zenithangs = np.deg2rad(np.array(solpos.loc[day].apparent_zenith))
    azimuthangs = np.deg2rad(np.array(solpos.loc[day].azimuth))
    invalidList = []
    for q in range(len(zenithangs)):
        if np.deg2rad(zenithangs[q])<0 or np.deg2rad(zenithangs[q])>(np.pi/2):
            invalidList.append(q)
    
    p_t = np.full(len(zenithangs),theta_p)
    p_p = np.full(len(zenithangs),phi_p)
 
    projList =  solar_panel_projection(zenithangs,azimuthangs,p_t,p_p)
    
    for index in invalidList:
        projList[index]=0
    for i in range(0,len(projList)):
        projList[i] = projList[i] * S0A

    
    return projList

def solar_elevation_angledeg(θ):
    return 90 - θ


def NewFluxMethod(zenithangs,azimuthangs, i_list, j_list):
    projList =  solar_panel_projection(zenithangs,azimuthangs,i_list,j_list)
    for i in range(0,len(projList)):
        if(zenithangs[i] <= 0 or zenithangs[i] >= np.pi/2):
            projList[i] = 0
        else:
            projList[i] = effectPerFlux*(projList[i] * S0A) if (projList[i] * S0A) <= maxFlux else maxEffect
    
    return ((integrate.simpson(projList,dx = 3600))*Area)/3600000

    
    return projList

def fluxToEffect(flux):
    result  = []
    for i in range (len(flux)): 
        result.append(effectPerFlux*flux[i] if flux[i]<=maxFlux else maxEffect)
       # result.append(flux[i]/maxFlux*maxEffect if flux[i]<=maxFlux else maxEffect)
    return result

def KwtPerDay(integral):
    L=1.755
    B=1.038
    J=integral*L*B
    return(J/3600000)

In [8]:
tidszone = "Europe/Copenhagen"
start_dato = "2024-01-01"
slut_dato = "2024-12-31"
delta_tid = "H"  # "Min", "H",

# Definition of Location object. Coordinates and elevation of Amager, Copenhagen (Denmark)
site = Location(
    55.7861111111, 12.5230555556, tidszone, 10, "DTU (DK)"
)  # latitude, longitude, time_zone, altitude, name

# Definition of a time range of simulation
times = pd.date_range(
    start_dato + " 00:00:00", slut_dato + " 23:59:00", inclusive="left", freq=delta_tid, tz=tidszone
)
# Estimate Solar Position with the 'Location' object
solpos = site.get_solarposition(times)

In [9]:
results = []

# Solens zenith og azimuth vinkler vinkler
zenithangs = np.deg2rad(np.array(solpos.loc[start_dato:slut_dato].apparent_zenith)) 
azimuthangs = np.deg2rad(np.array(solpos.loc[start_dato:slut_dato].azimuth))

# Panelets phi vinkel
phi_p = 180
p_p = np.full(len(zenithangs),np.deg2rad(phi_p))

theta_p = 39
p_t = np.full(len(zenithangs),np.deg2rad(theta_p))
    
# Beregner effekten per år for given vinkel
effectPerYear = NewFluxMethod(zenithangs,azimuthangs, p_t, p_p) 
print(effectPerYear)
results.append(effectPerYear)

23041.7710997019
